In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.externals import joblib
import warnings
warnings.filterwarnings('ignore')

In [7]:
train_df=pd.read_excel("Food_Train.xlsx")
test_df=pd.read_excel("Food_Test.xlsx")
rstr=train_df.append(test_df)

In [8]:
rstr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16921 entries, 0 to 4230
Data columns (total 9 columns):
CITY             16774 non-null object
COST             12690 non-null float64
CUISINES         16921 non-null object
LOCALITY         16793 non-null object
RATING           16917 non-null object
RESTAURANT_ID    16921 non-null int64
TIME             16921 non-null object
TITLE            16921 non-null object
VOTES            15315 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 1.3+ MB


In [9]:
rstr.describe()

,COST,RESTAURANT_ID
count,12690.000000,16921.000000
mean,655.252246,7767.211631
std,627.003540,4493.287626
min,20.000000,0.000000
25%,300.000000,3873.000000
50%,500.000000,7751.000000
75%,800.000000,11672.000000
max,14000.000000,15573.000000


In [10]:
rstr.duplicated().sum()

26

In [11]:
rstr=rstr.drop_duplicates()

In [12]:
rstr.isnull().sum()

CITY              147
COST             4230
CUISINES            0
LOCALITY          128
RATING              4
RESTAURANT_ID       0
TIME                0
TITLE               0
VOTES            1602
dtype: int64

In [13]:
rstr['Location']=rstr['CITY']+' '+rstr['LOCALITY']
rstr.drop(columns=['CITY','LOCALITY'])

,COST,CUISINES,RATING,RESTAURANT_ID,TIME,TITLE,VOTES,Location
0,1200.0,"Malwani, Goan, North Indian",3.6,9438,"11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",CASUAL DINING,49 votes,Thane Dombivali East
1,1500.0,"Asian, Modern Indian, Japanese",4.2,13198,6pm – 11pm (Mon-Sun),"CASUAL DINING,BAR",30 votes,Chennai Ramapuram
2,800.0,"North Indian, Chinese, Biryani, Hyderabadi",3.8,10915,"11am – 3:30pm, 7pm – 11pm (Mon-Sun)",CASUAL DINING,221 votes,Chennai Saligramam
3,800.0,"Tibetan, Chinese",4.1,6346,11:30am – 1am (Mon-Sun),QUICK BITES,24 votes,Mumbai Bandra West
4,300.0,Desserts,3.8,15387,11am – 1am (Mon-Sun),DESSERT PARLOR,165 votes,Mumbai Lower Parel
...,...,...,...,...,...,...,...,...
4226,NaN,"North Indian, Mughlai, Chinese",3.9,9057,11:30am – 11:30pm (Mon-Sun),CASUAL DINING,287 votes,New Delhi Punjabi Bagh
4227,NaN,"Biryani, North Indian, Sandwich, Salad, Wraps",4.3,1247,11am – 1am (Mon-Sun),None,469 votes,Bangalore HSR Layout
4228,NaN,"Continental, North Indian",3.7,8617,9:30am – 10:30pm (Mon-Sun),QUICK BITES,53 votes,Faridabad Sector 86
4229,NaN,"Rolls, Beverages",-,6485,"11am – 11:30pm (Mon, Tue, Wed, Thu, Sat, Sun),...",QUICK BITES,NaN,Kochi Kochi


In [14]:
rstr.dropna(subset=['Location'],inplace=True)

In [15]:
rstr

,CITY,COST,CUISINES,LOCALITY,RATING,RESTAURANT_ID,TIME,TITLE,VOTES,Location
0,Thane,1200.0,"Malwani, Goan, North Indian",Dombivali East,3.6,9438,"11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",CASUAL DINING,49 votes,Thane Dombivali East
1,Chennai,1500.0,"Asian, Modern Indian, Japanese",Ramapuram,4.2,13198,6pm – 11pm (Mon-Sun),"CASUAL DINING,BAR",30 votes,Chennai Ramapuram
2,Chennai,800.0,"North Indian, Chinese, Biryani, Hyderabadi",Saligramam,3.8,10915,"11am – 3:30pm, 7pm – 11pm (Mon-Sun)",CASUAL DINING,221 votes,Chennai Saligramam
3,Mumbai,800.0,"Tibetan, Chinese",Bandra West,4.1,6346,11:30am – 1am (Mon-Sun),QUICK BITES,24 votes,Mumbai Bandra West
4,Mumbai,300.0,Desserts,Lower Parel,3.8,15387,11am – 1am (Mon-Sun),DESSERT PARLOR,165 votes,Mumbai Lower Parel
...,...,...,...,...,...,...,...,...,...,...
4226,New Delhi,NaN,"North Indian, Mughlai, Chinese",Punjabi Bagh,3.9,9057,11:30am – 11:30pm (Mon-Sun),CASUAL DINING,287 votes,New Delhi Punjabi Bagh
4227,Bangalore,NaN,"Biryani, North Indian, Sandwich, Salad, Wraps",HSR Layout,4.3,1247,11am – 1am (Mon-Sun),None,469 votes,Bangalore HSR Layout
4228,Faridabad,NaN,"Continental, North Indian",Sector 86,3.7,8617,9:30am – 10:30pm (Mon-Sun),QUICK BITES,53 votes,Faridabad Sector 86
4229,Kochi,NaN,"Rolls, Beverages",Kochi,-,6485,"11am – 11:30pm (Mon, Tue, Wed, Thu, Sat, Sun),...",QUICK BITES,NaN,Kochi Kochi


In [16]:
from fuzzywuzzy import process
 
names_array=[]
def match_names(wrong_names,correct_names):
    for row in wrong_names:
        x=process.extractOne(row, correct_names)
        if x[1]<60:
            names_array.append('Others')
        else:
            names_array.append(x[0])
    return names_array

In [17]:
correct_names=['Bangalore','Thane',
'Hyderabad','Andheri',
'Delhi', 'Kerala',
'Chennai', 'Bandra',
'Mumbai', 'Telangana',
'Kochi', 
'Noida', 
'Gurgaon', 'Ernakulam',
'Faridabad', 'Ghaziabad',
'Secunderabad' ]
name_match=match_names(rstr.Location,correct_names)    

print(len(names_array))
rstr['Location']=names_array

16747


In [18]:
cuisines_list=[]
for row in rstr['CUISINES']:
    cuisines_list.append(list(row.split(',')))

rstr['CUISINES']=cuisines_list

In [19]:
rstr['CUISINES'].isna().sum()

0

In [20]:
rstr_cuisines=rstr['CUISINES'].apply(lambda x: pd.Series(1, x))

In [21]:
title_list=[]
for row in rstr['TITLE']:
    title_list.append(list(row.split(',')))
rstr['TITLE']=title_list

In [22]:
rstr_title=rstr['TITLE'].apply(lambda x: pd.Series(1, x))

In [23]:
rstr_title.head()

,CASUAL DINING,BAR,QUICK BITES,DESSERT PARLOR,CAFÉ,MICROBREWERY,BEVERAGE SHOP,IRANI CAFE,BAKERY,None,...,FOOD TRUCK,MESS,KIOSK,CLUB,CONFECTIONERY,DHABA,MEAT SHOP,COCKTAIL BAR,PAAN SHOP,BHOJANALYA
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
rstr[rstr['RATING'].isna()]

,CITY,COST,CUISINES,LOCALITY,RATING,RESTAURANT_ID,TIME,TITLE,VOTES,Location
204,Kochi,200.0,[Bakery],Marine Drive,NaN,15062,Not Available,[BAKERY],NaN,Kochi
421,Hyderabad,350.0,"[Cafe, Fast Food]",Banjara Hills,NaN,14127,Not Available,[CAFÉ],NaN,Hyderabad
2035,Kochi,NaN,"[Kerala, Arabian]",Kakkanad,NaN,2776,Not Available,[QUICK BITES],NaN,Kochi
2758,Kochi,NaN,"[Chinese, North Indian, South Indian]",Tripunithura,NaN,11301,Not Available,[None],NaN,Kochi


In [25]:
rstr["RATING"] = rstr.groupby("CITY").RATING.transform(lambda x : x.fillna(x.mode()[0]))

In [26]:
rstr['RATING']=rstr['RATING'].str.extract('(\d+)').astype(float)

In [27]:
rstr['VOTES'].isna().sum()

1579

In [28]:
rstr.VOTES.fillna('0',inplace=True)
rstr['VOTES']=rstr['VOTES'].str.extract('(\d+)').astype(float)

In [29]:
rstr.drop(columns='CITY',inplace=True)
rstr.drop(columns='LOCALITY',inplace=True)
rstr.drop(columns='CUISINES',inplace=True)

In [30]:
rstr.head()

,COST,RATING,RESTAURANT_ID,TIME,TITLE,VOTES,Location
0,1200.0,3.0,9438,"11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",[CASUAL DINING],49.0,Thane
1,1500.0,4.0,13198,6pm – 11pm (Mon-Sun),"[CASUAL DINING, BAR]",30.0,Chennai
2,800.0,3.0,10915,"11am – 3:30pm, 7pm – 11pm (Mon-Sun)",[CASUAL DINING],221.0,Chennai
3,800.0,4.0,6346,11:30am – 1am (Mon-Sun),[QUICK BITES],24.0,Bandra
4,300.0,3.0,15387,11am – 1am (Mon-Sun),[DESSERT PARLOR],165.0,Mumbai


In [32]:
rstr_City=pd.get_dummies(rstr['Location'])
rstr.drop(columns='Location',inplace=True)
rstr_City.head()

,Andheri,Bandra,Bangalore,Chennai,Delhi,Ernakulam,Faridabad,Ghaziabad,Gurgaon,Hyderabad,Kerala,Kochi,Mumbai,Noida,Others,Secunderabad,Telangana,Thane
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [33]:
rstr = pd.concat([rstr,rstr_City,rstr_cuisines,rstr_title], axis=1)

In [34]:
rstr.head()

,COST,RATING,RESTAURANT_ID,TIME,TITLE,VOTES,Andheri,Bandra,Bangalore,Chennai,...,FOOD TRUCK,MESS,KIOSK,CLUB,CONFECTIONERY,DHABA,MEAT SHOP,COCKTAIL BAR,PAAN SHOP,BHOJANALYA
0,1200.0,3.0,9438,"11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",[CASUAL DINING],49.0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1500.0,4.0,13198,6pm – 11pm (Mon-Sun),"[CASUAL DINING, BAR]",30.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,800.0,3.0,10915,"11am – 3:30pm, 7pm – 11pm (Mon-Sun)",[CASUAL DINING],221.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,800.0,4.0,6346,11:30am – 1am (Mon-Sun),[QUICK BITES],24.0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,300.0,3.0,15387,11am – 1am (Mon-Sun),[DESSERT PARLOR],165.0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
rstr.drop(columns='TITLE',inplace=True)

In [36]:
rstr_column_category = rstr.select_dtypes(exclude=np.number).columns
rstr_column_category

Index(['TIME'], dtype='object')

In [37]:
rstr.drop(columns='TIME',inplace=True)

In [42]:
rstr.fillna(0,inplace=True)

In [44]:
rstr.head()

,COST,RATING,RESTAURANT_ID,VOTES,Andheri,Bandra,Bangalore,Chennai,Delhi,Ernakulam,...,FOOD TRUCK,MESS,KIOSK,CLUB,CONFECTIONERY,DHABA,MEAT SHOP,COCKTAIL BAR,PAAN SHOP,BHOJANALYA
0,1200.0,3.0,9438,49.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1500.0,4.0,13198,30.0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,800.0,3.0,10915,221.0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,800.0,4.0,6346,24.0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,300.0,3.0,15387,165.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
rstr.tail()

,COST,RATING,RESTAURANT_ID,VOTES,Andheri,Bandra,Bangalore,Chennai,Delhi,Ernakulam,...,FOOD TRUCK,MESS,KIOSK,CLUB,CONFECTIONERY,DHABA,MEAT SHOP,COCKTAIL BAR,PAAN SHOP,BHOJANALYA
4226,0.0,3.0,9057,287.0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4227,0.0,4.0,1247,469.0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4228,0.0,3.0,8617,53.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4229,0.0,0.0,6485,0.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4230,0.0,3.0,2421,63.0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
rstr.shape

(16747, 276)

In [87]:
rstr.fillna(0,inplace=True)
rstr.isna().sum()

COST             0
RATING           0
RESTAURANT_ID    0
VOTES            0
Andheri          0
                ..
DHABA            0
MEAT SHOP        0
COCKTAIL BAR     0
PAAN SHOP        0
BHOJANALYA       0
Length: 276, dtype: int64

In [88]:
rstr.head()

,COST,RATING,RESTAURANT_ID,VOTES,Andheri,Bandra,Bangalore,Chennai,Delhi,Ernakulam,...,FOOD TRUCK,MESS,KIOSK,CLUB,CONFECTIONERY,DHABA,MEAT SHOP,COCKTAIL BAR,PAAN SHOP,BHOJANALYA
0,1200.0,3.0,9438,49.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1500.0,4.0,13198,30.0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,800.0,3.0,10915,221.0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,800.0,4.0,6346,24.0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,300.0,3.0,15387,165.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
X = rstr.drop('COST', axis = 1)
y = rstr['COST']

In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=75)

In [66]:
X_train.shape

(11722, 275)

In [67]:
X_test.shape

(5025, 275)

In [68]:
y_train.shape

(11722,)

In [69]:
y_test.shape

(5025,)

In [104]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)


y_train = y_train.reshape((len(y_train), 1))

y_train = sc.fit_transform(y_train)

y_train = y_train.ravel()

In [105]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
dtrain_predictions = model.predict(X_train)

In [106]:
from sklearn.model_selection import cross_val_score
a = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

In [107]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_squared_log_error, r2_score
print("\nModel Report")
print("RMSE : %.4g" % np.sqrt(mean_squared_error(y_train, dtrain_predictions)))
    



Model Report
RMSE : 0.744


In [109]:
y_test["res_linear"] =  model.predict(X_test)

In [110]:
print('r2 train',r2_score(y_train,dtrain_predictions))

r2 train 0.44645559491949993


In [114]:
from sklearn.ensemble import GradientBoostingRegressor

gbr=GradientBoostingRegressor( loss = 'huber',learning_rate=0.001,n_estimators=350, max_depth=6
                              ,subsample=1,
                              verbose=False,random_state=126)   # Leaderboard SCORE :  0.8364249755816828 @ RS =126 ,n_estimators=350, max_depth=6

gbr.fit(X_train,y_train)

y_pred_gbr = sc.inverse_transform(gbr.predict(X_test))
y_pred_gbr

array([-0.28120608, -0.00586583, -0.27897371, ..., -0.03231196,
       -0.26993038, -0.15209958])

In [115]:
y_pred_gbr = pd.DataFrame(y_pred_gbr, columns = ['COST']) # Converting to dataframe
print(y_pred_gbr)

          COST
0    -0.281206
1    -0.005866
2    -0.278974
3     0.546925
4    -0.278974
...        ...
5020 -0.269930
5021 -0.281206
5022 -0.032312
5023 -0.269930
5024 -0.152100

[5025 rows x 1 columns]
